# Climate policy diffusion 
- This script supports the analysis in the 'Climate policy diffusion' project. 
- This analysis seeks to uncover a link between the 'international environment' and the adoption of climate policy. In particular, it is interested in identifying (i) salient 'factors' and (ii) diffusion channels.
- It is focused exclusively on policy adoption in the power sector of national jurisdictions
- Each set of estimated models focuses on one specific policy:
    - Emissions Trading Systems (pricing mechanism)
    - Carbon Taxes (pricing mechanism)
    - Feed in Tariffs
    - Renewable Energy Quotas (non-price mechanism)
    
POLICY
Bilateral trade could capture general bilateral economic integration - in which case bilat x policy ==> signal; (making a case for bilat trade to affect technology diffusion would require to work at a disaggregated level, at the very least)

"Bilateral" patenting activity could capture technological diffusion;

TECHNOLOGY
It's about access to technology more than 'diffusion' per se - so one can test global vs 'weighted' technological pools
Like nodes in a network

In [1]:
import csv
import pprint
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re

import numpy.linalg as LA

## Just making the plots look better
mpl.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['font.size'] = 12

path_cp_diff = "/Users/gd/GitHub/ClimatePolicyDiffusion/_dataset"
path_raw_data = "/Users/gd/OneDrive - rff/Documents/Research/projects/climate_policy_adoption/cp_diffusion/data/raw_data"

In [2]:
def concatenate(indir):#,outfile):
    os.chdir(indir) #sets the current directory to 'indir'
    fileList=glob.glob("*.csv") #this command generates a list of csv files
    dfList = []

    #each iteration of the loop will add a dataframe to the list
    for filename in fileList:
        df=pd.read_csv(filename, header=0, encoding='latin-1')
        dfList.append(df)
    concatDf=pd.concat(dfList,axis=0) #'axis=0' means that we are concatenating vertically, if we wanted to concatenate horizontally, we should use 'axis=1'

    return concatDf

map_iea_wb = {"CÃ\x83Â´te d'Ivoire": "Cote d'Ivoire", "CÃ´te d'Ivoire": "Cote d'Ivoire",
              '"China (P.R. of China and Hong Kong, China)"': 'China (P.R. of China and Hong Kong, China)',
              "People's Republic of China": 'China', 'CuraÃ\x83Â§ao/Netherlands Antilles': 'Curacao/Netherlands Antilles',
              'CuraÃ§ao': 'Curacao', 'CuraÃ§ao/Netherlands Antilles': 'Curacao/Netherlands Antilles',
              'Democratic Republic of Congo': 'Congo, Dem. Rep.', 'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
              'Republic of the Congo': 'Congo, Rep.', 'Egypt': 'Egypt, Arab Rep.', 'Hong Kong (China)': 'Hong Kong SAR, China',
              'Islamic Republic of Iran': 'Iran, Islamic Rep.', "Democratic People's Republic of Korea": 'Korea, Dem. Rep.',
              'Korea': 'Korea, Rep.', 'Kyrgyzstan': 'Kyrgyz Republic', 'Republic of North Macedonia': 'North Macedonia',
              'Republic of Moldova':'Moldova', 'Chinese Taipei':'Taiwan, China',
              'Venezuela': 'Venezuela, RB', 'Plurinational State of Bolivia':'Bolivia',
              'United Republic of Tanzania':'Tanzania',
              'Bolivarian Republic of Venezuela': 'Venezuela, RB', 'Viet Nam': 'Vietnam', 'Yemen': 'Yemen, Rep.'}

# I. Policy variables (binary)

In [3]:
# Carbon pricing policies

indir = "/Users/gd/GitHub/WorldCarbonPricingDatabase/_dataset/data/CO2/national"
cp = concatenate(indir)
cp = cp[["jurisdiction", "year", "ipcc_code", "Product", "tax", "ets"]]

# select power sector only
cp = cp[cp["ipcc_code"].isin(["1A1A1"])]
cp.drop("ipcc_code", axis=1, inplace=True)
cp.rename(columns={"jurisdiction":"Country"}, inplace=True)

# aggregate across fuels (product)
cp = cp.groupby(by=["Country", "year"]).sum()
cp.reset_index(inplace=True)

# set value to 1 if > 0
cp.loc[cp["tax"] > 0, "tax"] = 1
cp.loc[cp["ets"] > 0, "ets"] = 1


#FiT and Renewable Portfolio Standards

fit = pd.read_excel(path_raw_data+"/FiT_RPS/FiT_RPS_adoption.xlsx", sheet_name="FiT")
rps = pd.read_excel(path_raw_data+"/FiT_RPS/FiT_RPS_adoption.xlsx", sheet_name="RPS")

fit = fit.melt(id_vars="Country")
fit.columns = ["Country", "year", "FiT"]
rps = rps.melt(id_vars="Country")
rps.columns = ["Country", "year", "RPS"]

rep = fit.merge(rps, on=["Country", "year"])

# policy indicators dataframe
policies = cp.merge(rep, on=["Country", "year"], how="inner")
policies[["FiT", "RPS"]] = policies[["FiT", "RPS"]].astype(float)

In [4]:
# these loops creates the indicator values for carbon taxes, emissions trading FiT, RPS based on first year of adoption 
# (as currently written, it assumes that, once implemented, policies are not repealed)
for policy in ["tax", "ets", "FiT", "RPS"]:
    for ctry in policies.Country.unique():
        value = 0

        for yr in policies.year.unique():
            if policies.loc[(policies.Country==ctry) & (policies.year==yr), policy].item() == 1:
                value = 1
            policies.loc[(policies.Country==ctry) & (policies.year==yr), policy] = value

# policy aggregate indicators
policies.loc[:, "Pricing"] = policies.loc[:, "tax"] + policies.loc[:, "ets"] 
policies.loc[:, "Tech"] = policies.loc[:, "FiT"] + policies.loc[:, "RPS"] 

policies.loc[policies.Pricing > 0, "Pricing"] = 1
policies.loc[policies.Tech > 0, "Tech"] = 1

policies.columns = ["Country", "year", "tax", "ets", "fit", "rps", "pricing", "techpol"]

# II. Technology proxies

Proxies: 
- patents granted at the EPO for climate change mitigation technologies in electricity generation, transmission and distribution
- installed wind and solar generation capacity

In [5]:
#change names in OECD PAT file
patents = pd.read_csv(path_raw_data+'/OECD/Patents/PATS_IPC_05022023173900507.csv')
patents.rename(columns={"Value":"Patent_count"}, inplace=True)

#select only "inventors" (following Lovely & Popp) and the patent family 
patents = patents.loc[(patents.KINDCOUNTRY=="INVENTORS") & (patents.KINDPATENT=="EPO_A") & (patents.KINDDATE=="PRIORITY"), :]

patents.drop(['KINDCOUNTRY', 'Reference country', 'KINDPATENT', 'Patents Office & Patents Families', 'LOCATION', 'Technology domains & IPC', 
              'KINDDATE', 'PowerCode Code', 'PowerCode', 'IPC', 'TIME', 'Unit Code', 'Unit',
              'Reference Date', 'Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags'], axis=1, inplace=True)

recap = pd.read_csv(path_raw_data+"/RE capacity/RE_all.csv")
recap.columns = ["Country", "year", "capacity_ws"]

elecprod = pd.read_csv(path_raw_data+'/IEA/IEA_Elec_prod.csv', usecols = ["Country", "Product", "Time", "Value"])
elecprod = elecprod.groupby(by=["Country", "Time"]).sum()
elecprod.reset_index(inplace=True)

oecd_wb_map = {"China (People's Republic of)": 'China', 'Egypt': 'Egypt, Arab Rep.', 'Iran': 'Iran, Islamic Rep.',
               'Russia': 'Russian Federation', 'Venezuela': 'Venezuela, RB'}

patents['Country'] = patents['Country'].replace(to_replace=oecd_wb_map)

# The patent series presents a break (i.e., missing data) in 2017 for the following countries: 
countries2017 = ['Algeria', 'Belarus', 'Bermuda', 'Bosnia and Herzegovina', 'Cayman Islands', 
                 'Costa Rica', 'Cuba', 'Cyprus', "Democratic People's Republic of Korea", 'Djibouti', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Georgia', 'Guatemala', 
                 'Iran, Islamic Rep.', 'Jamaica', 'Kenya', 'Kuwait', 'Lebanon', 'North Macedonia', 'Moldova', 'Mongolia', 'Nigeria', 'Peru', 'Philippines', 'Puerto Rico', 'Seychelles', 
                 'Sri Lanka', 'Trinidad and Tobago', 'Tunisia', 'Uruguay', 'Venezuela, RB', 'Zimbabwe']

# Since data for 2019 is part of the series, I create missing entries (rows) and interpolate values
for ctry in countries2017:
    temp_df = pd.DataFrame(data={"Country":[ctry], "Time":[2017], "Patent_count":[np.nan]})
    patents = pd.concat([patents, temp_df])

patents.sort_values(by=["Country", "Time"], inplace=True)
patents["Patent_count"].interpolate(inplace=True)



## II.1 Net installed capacity stock

In [6]:
ctries = [x for x in sorted(recap.Country.unique()) if x not in ["St. Pierre-Miquelon", "EU 28"]]
yrs = sorted(recap.year.unique())

# generating a dataframe with all years for all countries
df = pd.DataFrame(columns=["Country", "year"])

for ctry in ctries:
    for yr in yrs:
        df2 = pd.DataFrame([[ctry, yr]], columns=["Country", "year"])
        df = df.append(df2, ignore_index=True)

df = df.merge(recap, on=["Country", "year"], how="left")
df.year = df.year.astype(int)

# Imputing missing net installed capacity data

# filling nan entries of column 'capacity_ws' with zeros if the first year for which we have data is strictly lower than 15
# This makes two assumptions:
#    1. that no installed capacity was added AND then retired prior to first year for which data is available
#    2. that the capacity was installed in that first year for which data was available
# unlikely to introduce many inaccuracies in the series as 
#    1. it starts in 1990 and data is available from 2000 onward at the latest
#    2. most countries had a an installed capacity of '0' in the first years for which data is available
# The countries for which it might be problematic are: 'Costa Rica' (1995, 17), 'Germany' (1991, 112), 'Brazil' (2000, 22), 
# 'China' (2000, 375), 'Iran, Islamic Rep.' (2000, 11), 'Morocco' (2000, 61)
# For these countries, net capacity data does not go back to 1990 and capacity in the first year for which data is available exceeds 10 Mw
# a = recap.drop_duplicates(subset=["Country"], keep='first')
# a.loc[(a.capacity_ws != 0) & (a.Year > 1990) & (a.capacity_ws > 10), :]

for ctry in ctries:
    for i in range(0, 30):
        j = 2019 - i
        
        is_loc = (df.Country==ctry) & (df.year==j)
        
        if pd.isna(df.loc[is_loc, "capacity_ws"].item()) == True:
            if df.loc[(df.Country==ctry) & (df.year==j+1), "capacity_ws"].item() < 15:
                df.loc[is_loc, "capacity_ws"] = 0
    
# CALCULATING NET CAPACITY ADDITIONS (capacity additions - capacity retirement)           

for ctry in ctries:
    
    #Assume capacity was added in 1990 if capacity_ws in 1990 < 15 Mw
    if df.loc[(df.Country==ctry) & (df.year==1990), "capacity_ws"].item() < 15:
        df.loc[(df.Country==ctry) & (df.year==1990), "capacity_ws_add"] = df.loc[(df.Country==ctry) & (df.year==1990), "capacity_ws"]
    
    for i in range(1991, 2020):
        is_loc = (df.Country==ctry) & (df.year==i)
        df.loc[is_loc, "capacity_ws_add"] = df.loc[is_loc, "capacity_ws"].item() - df.loc[(df.Country==ctry) & (df.year==i-1), "capacity_ws"].item()

#if gross capacity at end of year ('capacity_ws') is naught, then we know that no capacity was added in that year and hence 'capacity_ws_add' is 0
df.loc[df.capacity_ws==0, "capacity_ws_add"] = 0

/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/3040474877.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)
/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/3040474877.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)
/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/3040474877.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)
/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/3040474877.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.app

In [7]:

# NET CAPACITY STOCK

disc_rate = 0.04

recap_cum = pd.DataFrame()

for ctry in df.Country.unique():

    temp_cap = df.loc[df.Country==ctry, :].copy()
    temp_cap.reset_index(inplace=True)
    temp_cap.drop("index", axis=1, inplace=True)
    temp_cap["discount_stock"] = np.nan
    
    # Following from assumption above, if capacity_ws in 1990 < 15 Mw, it is assumed that it was added in 1990;
    # Hence the discounted sctock is equal to capacity_ws_add
    if temp_cap.loc[0, "capacity_ws_add"] < 15:
        temp_cap.loc[0, "discount_stock"] = temp_cap.loc[0, "capacity_ws_add"]
        
        for i in range(1, len(temp_cap)):
            temp_cap.loc[i, "discount_stock"] = temp_cap.loc[i-1, "discount_stock"]/(1+disc_rate) + temp_cap.loc[i, "capacity_ws_add"]

    # for this group of countries, we know the first year of renewable production, thanks to IEA production data
    # we use that year together with actual production data to estimate initial stock 
    elif ctry in elecprod.Country.unique():
        
        temp_prod = elecprod.loc[elecprod.Country==ctry,:].copy()
        temp_prod.reset_index(inplace=True)
        temp_prod.drop("index", axis=1, inplace=True)
        temp_prod["gr_rate"] = np.nan
        
        first_addition_index = temp_cap.capacity_ws_add.first_valid_index()
        first_addition_year = temp_cap.loc[first_addition_index, "year"]

        first_production_index = temp_prod.Value.loc[temp_prod.Value!=0].index[0].item()
        first_production_year = temp_prod.loc[first_production_index, "Time"]
        
        for i in range(first_production_index+1, len(temp_prod)):
            temp_prod.loc[i, "gr_rate"] = (temp_prod.loc[i, "Value"]-temp_prod.loc[i-1, "Value"])/temp_prod.loc[i-1, "Value"]
        
        # calculate the ratio in second year to make sure capacity is operational and generation value > 0 
        ratio = temp_prod.loc[temp_prod.Time==first_addition_year+1, "Value"].item()/temp_cap.loc[temp_cap.year == first_addition_year+1, "capacity_ws"].item()
        
        cap_first_year = temp_prod.loc[temp_prod.Time==first_production_year, "Value"].item()/ratio
        
        temp_prod.loc[temp_prod.Time==first_production_year, "capacity_ws"] = cap_first_year
        
        for yr in range(first_production_year+1, first_addition_year+1):
            temp_prod.loc[temp_prod.Time==yr, "capacity_ws"] = temp_prod.loc[temp_prod.Time==yr-1, "capacity_ws"].item()*(1+temp_prod.loc[temp_prod.Time==yr, "gr_rate"].item())
        
        temp_prod["capacity_ws_add"] = np.nan
        temp_prod.loc[temp_prod.Time==first_production_year, "capacity_ws_add"] = temp_prod.loc[temp_prod.Time==first_production_year, "capacity_ws"]
        
        for yr in range(first_production_year+1, first_addition_year+1):
            temp_prod.loc[temp_prod.Time==yr, "capacity_ws_add"] = temp_prod.loc[temp_prod.Time==yr, "capacity_ws"].item() - temp_prod.loc[temp_prod.Time==yr-1, "capacity_ws"].item()
            
        temp_prod["capacity_ws_ds"] = np.nan
        temp_prod.loc[temp_prod.Time==first_production_year, "capacity_ws_ds"] = temp_prod.loc[temp_prod.Time==first_production_year, "capacity_ws"]
        
        # NET STOCK
        for yr in range(first_production_year+1, first_addition_year+1):
            temp_prod.loc[temp_prod.Time==yr, "capacity_ws_ds"] = temp_prod.loc[temp_prod.Time==yr-1, "capacity_ws_ds"].item()/(1+disc_rate) + temp_prod.loc[temp_prod.Time==yr, "capacity_ws_add"].item()
                
        temp_cap.loc[temp_cap.year==first_addition_year-1, "discount_stock"] = temp_prod.loc[temp_prod.Time==first_addition_year-1, "capacity_ws_ds"].item()
        
        first_ds_index = temp_cap.discount_stock.first_valid_index()
        
        for i in range(first_ds_index+1, len(temp_cap)):
            temp_cap.loc[i, "discount_stock"] = temp_cap.loc[i-1, "discount_stock"]/(1+disc_rate) + temp_cap.loc[i, "capacity_ws_add"]
    
#    elif ctry in ['Brazil', 'China', 'Costa Rica', 'India', 'Morocco']:
    
    recap_cum = pd.concat([recap_cum, temp_cap])

recap_cum.drop(["capacity_ws_add", "capacity_ws"], axis=1, inplace=True)
recap_cum.columns = ["Country", "year", "capacity_ws_ds"]

## II.2 Discounted cumulative patents stock

In [8]:
patents_cum = pd.DataFrame()

for ctry in patents.Country.unique():
    temp = patents.loc[patents.Country==ctry, :].copy()
    temp.reset_index(inplace=True)
    temp.drop("index", axis=1, inplace=True)
    temp["discount_stock"] = np.nan
    
    # initializing discounted stock at value of first patent addition
    temp.loc[0, "discount_stock"] = temp.loc[0, "Patent_count"]
    
    for i in range(1, len(temp)):
        temp.loc[i, "discount_stock"] = temp.loc[i-1, "discount_stock"]/1.1 + temp.loc[i, "Patent_count"]
    
    patents_cum = pd.concat([patents_cum, temp])

patents_cum.drop("Patent_count", axis=1, inplace=True)
patents_cum.columns = ["Country", "year", "patents_ds"]

# II. Bilateral (dyadic) weights
## II.1 International trade

In [9]:
os.chdir(path_raw_data+'/bilateral_trade/IMF_DoT_Stats') #sets the current directory to 'indir'
fileList=glob.glob("*.csv") #this command generates a list of csv files
# to concatenate, we will stack the files into a single Python list
# the method will generate a single output file as output
# before starting the loop, we need to create an empty list object

dfList = []

colnames = ["REPORTER_COUNTRY","Reporter Country","FLOW","Flow","LOCATION","Partner Country","FREQUENCY","Frequency","TIME","Time","Value","Flag Codes","Flags"]

#each iteration of the loop will add a dataframe to the list
for filename in fileList:
    #print(filename)
    df=pd.read_csv(filename, header=0)
    dfList.append(df)
concatDf=pd.concat(dfList,axis=0) #'axis=0' means that we are concatenating vertically, if we wanted to concatenate horizontally, we should use 'axis=1'
concatDf.columns=colnames

In [10]:
bilat_trade = concatDf.rename(columns={'Reporter Country': 'Reporter_Country', 'Partner Country': 'Partner_Country'})

imf_wb_map = {'Afghanistan, Islamic Republic of': 'Afghanistan', 'Armenia, Republic of': 'Armenia', 'Azerbaijan, Republic of': 'Azerbaijan',
              'Bahrain, Kingdom of': 'Bahrain', 'China, P.R.: Mainland': 'China', "CÃ\x83Â´te d'Ivoire": "Cote d'Ivoire", 'Curaçao':"Curacao",
              "CÃ´te d'Ivoire": "Cote d'Ivoire", "Côte d'Ivoire": "Cote d'Ivoire", 'The Bahamas': 'Bahamas, The', 'Brunei': 'Brunei Darussalam',
              'Congo, Democratic Republic of': 'Congo, Dem. Rep.', 'Congo, Republic of': 'Congo, Rep.', 'Egypt': 'Egypt, Arab Rep.',
              'The Gambia': 'Gambia, The', 'China, P.R.: Hong Kong': 'Hong Kong SAR, China', 'Iran, Islamic Republic of': 'Iran, Islamic Rep.',
              'Korea, Republic of': 'Korea, Rep.', "Korea, Democratic People's Republic of": 'Korea, Dem. Rep.',
              'Kosovo, Republic of': 'Kosovo', "Lao People's Democratic Republic": 'Lao PDR', 
              'Marshall Islands, Republic of': 'Marshall Islands', 'China, P.R.: Macao': 'Macao SAR, China', 
              "North Macedonia, Republic of":"North Macedonia", 'Russia': 'Russian Federation', 'São Tomé & Príncipe': 'Sao Tome and Principe',
              'Serbia, Republic of': 'Serbia', 'Timor-Leste, Democratic Republic of':'Timor-Leste',
              'Venezuela, Republica Bolivariana de': 'Venezuela, RB', 'Yemen, Republic of': 'Yemen, Rep.'}

# Replace names
bilat_trade['Reporter_Country'] = bilat_trade['Reporter_Country'].replace(to_replace=imf_wb_map)
bilat_trade['Partner_Country'] = bilat_trade['Partner_Country'].replace(to_replace=imf_wb_map)

# Drop non-country 'Reporter' and 'Partner'
excl_reporters = ["Sub-Saharan Africa"]

excl_partners = ['Advanced Economies','Africa','Africa not specified','Asia not specified','Countries&Areas, not specified','CIS',
            'Developing Asia','Emerging and Developing Asia','Emerging and Developing Economies','Emerging and Developing Europe',
            'Euro Area','Europe','European Union','Europe not specified','Export Earnings: Fuel','Export Earnings: Non-Fuel',
            'Middle East','Middle East not specified','Middle East, North Africa & Pakistan','Other Countries n.i.e.',
            'Sub-Saharan Africa (developing only)','Sub-Saharan Africa','Western Hemisphere','Western Hem. not specified','SACCA']

bilat_trade = bilat_trade.loc[(~bilat_trade.Partner_Country.isin(excl_partners)) & (~bilat_trade.Reporter_Country.isin(excl_reporters)), :] 
bilat_trade.drop(["REPORTER_COUNTRY", "LOCATION", "FREQUENCY", "Frequency", "Flow", "TIME", "Flag Codes", "Flags"], axis=1, inplace=True)

In [11]:
tradeflows = {}
tradeflows_share = {}
tradeflows_wld = {}

flow_names = {"TMG_CIF_USD":"IMP", "TXG_FOB_USD":"EXP", "TMG_CIF_USD+TMG_FOB_USD":"IMPEXP"}

imp_exp_bilat = bilat_trade.groupby(["Reporter_Country", "Partner_Country", "Time"]).sum()
imp_exp_bilat.reset_index(inplace=True)
imp_exp_bilat["FLOW"] = "TMG_CIF_USD+TMG_FOB_USD"

bilat_trade = pd.concat([bilat_trade, imp_exp_bilat])

for flow in ["TMG_CIF_USD", "TXG_FOB_USD", "TMG_CIF_USD+TMG_FOB_USD"]:
    tradeflows[flow_names[flow]] = bilat_trade[bilat_trade["FLOW"]==flow]
    tradeflows_wld[flow_names[flow]] = tradeflows[flow_names[flow]][tradeflows[flow_names[flow]]["Partner_Country"]=="World"]
    
    tradeflows_share[flow_names[flow]] = tradeflows[flow_names[flow]].merge(tradeflows_wld[flow_names[flow]], on=["Reporter_Country", "FLOW", "Time"])
    tradeflows_share[flow_names[flow]].rename(columns={"Value_x":"Value", "Value_y":"Total_year", "Partner_Country_x":"Partner_Country"}, inplace=True)
    tradeflows_share[flow_names[flow]].drop(["Partner_Country_y"], axis=1, inplace=True)

    tradeflows_share[flow_names[flow]]["Value_share_tot"] = tradeflows_share[flow_names[flow]].Value/tradeflows_share[flow_names[flow]].Total_year
    tradeflows_share[flow_names[flow]] = tradeflows_share[flow_names[flow]].loc[tradeflows_share[flow_names[flow]].Partner_Country != "World", :] #removing 'World' entry from partners' list
    tradeflows_share[flow_names[flow]].drop(["Value", "Total_year"], axis=1, inplace=True)
    tradeflows_share[flow_names[flow]].columns = ["Reporter", "FLOW", "Partner", "year", "trade_flow_share"]

In [12]:
# 25 'Partner' entities in the bilateral trade dataset are not present in the `policies` dataset.
# Some of them ('U.S.S.R.', 'Yugoslavia, SFR', 'Czechoslovakia', 'Serbia and Montenegro') are entities that have ceased to exist and 
# are now split into multiple countries. The `policies` dataset records entries for their successor entities.
# Remaining entities are simply absent from the `policies` dataset. However, this means that none of the policies were introduced in any of these entities. Hence
# we interpret absence from dataset as '0'.

# However none of these 'Partner' entities had a 
policy_trade_partners = set(tradeflows_share["IMP"].Partner.unique())-set(policies.Country.unique()).intersection(set(tradeflows_share["IMP"].Partner.unique()))


In [13]:
test = tradeflows_share[flow_names["TMG_CIF_USD"]]

east_asia = ["American Samoa", "Australia", "Brunei Darussalam", "Cambodia", "Fiji", "French Polynesia", "Guam", "Hong Kong SAR", 
             "Indonesia", "Japan", "Kiribati", "Korea, Dem. People's Rep.", "Korea, Rep.", "Lao PDR", "Macao SAR", 
             "Malaysia", "Marshall Islands", "Micronesia, Fed. Sts.", "Mongolia", "Myanmar", "Nauru", "New Caledonia", "New Zealand", "Northern Mariana Islands",
             "Palau", "Papua New Guinea", "Philippines", "Samoa", "Singapore", "Solomon Islands", "Thailand", "Timor-Leste", "Tonga", "Tuvalu", 
             "Vanuatu", "Vietnam"]

#test.loc[(test.Reporter.isin(east_asia)) & (test.Partner=="China") & (test.Year==2018)]#.describe()


## I.2. Shared institutional and cultural ties
### I.2.1. Membership of multilateral organisations

In [14]:
#EU

#0. List of all national jurisdictions - WB names, taken from PE name match file
country_list = ["Afghanistan","Albania","Algeria","Andorra","Angola","Antigua and Barbuda","Argentina","Armenia",
                "Australia","Austria","Azerbaijan","Bahamas, The","Bahrain","Bangladesh","Barbados","Belarus",
                "Belgium","Belize","Benin","Bhutan","Bolivia","Bosnia and Herzegovina","Botswana","Brazil",
                "Brunei Darussalam","Bulgaria","Burkina Faso","Burundi","Cabo Verde","Cambodia","Cameroon",
                "Canada","Central African Republic","Chad","Chile","China","Colombia","Comoros",                
                "Congo, Dem. Rep.","Congo, Rep.","Costa Rica","Cote d'Ivoire","Croatia","Cuba","Cyprus",
                "Czech Republic","Denmark","Djibouti","Dominica","Dominican Republic","Ecuador","Egypt, Arab Rep.",
                "El Salvador","Equatorial Guinea","Eritrea","Estonia","Ethiopia","Fiji","Finland","France","Gabon",
                "Gambia, The","Georgia","Germany","Ghana","Greece","Grenada","Guatemala","Guinea","Guinea-Bissau",
                "Guyana","Haiti","Honduras","Hong Kong SAR, China","Hungary","Iceland",
                "India","Indonesia","Iran, Islamic Rep.","Iraq","Ireland","Israel","Italy","Jamaica","Japan",
                "Jordan","Kazakhstan","Kenya","Kiribati","Korea, Dem. Rep.","Korea, Rep.","Kosovo","Kuwait",
                "Kyrgyz Republic","Lao PDR","Latvia","Lebanon","Lesotho","Liberia","Libya","Liechtenstein",
                "Lithuania","Luxembourg","Macao SAR, China","North Macedonia", "Madagascar","Malawi",
                "Malaysia","Maldives","Mali","Malta","Marshall Islands","Mauritania","Mauritius","Mexico",
                "Moldova","Monaco","Mongolia","Montenegro","Morocco","Mozambique","Myanmar","Namibia","Nepal",
                "Netherlands","New Zealand","Nicaragua","Niger","Nigeria","Norway","Oman","Pakistan","Palau",
                "Panama","Papua New Guinea","Paraguay","Peru","Philippines","Poland","Portugal","Puerto Rico",
                "Qatar","Romania","Russian Federation","Rwanda","Samoa","San Marino","Sao Tome and Principe",
                "Saudi Arabia","Senegal","Serbia","Seychelles","Sierra Leone","Singapore","Slovak Republic",
                "Slovenia","Solomon Islands","Somalia","South Africa","South Sudan","Spain","Sri Lanka",
                "St. Kitts and Nevis","St. Lucia","St. Vincent and the Grenadines","Sudan","Suriname","Swaziland",
                "Sweden","Switzerland","Syrian Arab Republic","Tajikistan","Tanzania","Thailand","Timor-Leste",
                "Togo","Tonga","Trinidad and Tobago","Tunisia","Turkey","Turkmenistan","Tuvalu","Uganda",
                "Ukraine","United Arab Emirates","United Kingdom","United States","Uruguay","Uzbekistan","Vanuatu",
                "Venezuela, RB","Vietnam","West Bank and Gaza","Yemen, Rep.","Zambia","Zimbabwe"]

years = [str(i) for i in range(1990,2019)]

#1. List of EU member states in each year
eu_list = ['Belgium','France','Netherlands','Germany','Italy','Luxembourg',
          'Denmark','Ireland','United Kingdom','Greece','Spain','Portugal',
          'Austria','Sweden','Finland','Czech Republic','Slovenia','Poland',
          'Slovakia','Estonia','Cyprus','Latvia','Lithuania','Malta','Hungary',
          'Romania','Bulgaria']

eu_dic = {'Belgium':1957,'France':1957,'Netherlands':1957,'Germany':1957,'Italy':1957,'Luxembourg':1957,
          'Denmark':1973,'Ireland':1973,'United Kingdom':1973,'Greece':1981,'Spain':1986,'Portugal':1986,
          'Austria':1995,'Sweden':1995,'Finland':1995,'Czech Republic':2004,'Slovenia':2004,'Poland':2004,
          'Slovakia':2004,'Estonia':2004,'Cyprus':2004,'Latvia':2004,'Lithuania':2004,'Malta':2004,'Hungary':2004,
          'Romania':2007,'Bulgaria':2007}

eu_dic_II = {}

for country in eu_list:
    for year in years:
        if int(year) >= eu_dic[country]:
            if year not in eu_dic_II:
                eu_dic_II[year] = [] 
            eu_dic_II[year].append(country)


eu_mat = {}


for reporter in country_list:
    if reporter not in eu_mat:
        eu_mat[reporter] = {}
    for partner in country_list:
        if partner not in eu_mat[reporter]:
            eu_mat[reporter][partner] = {}
        if partner != reporter:
            for year in years:
                if year not in eu_mat[reporter][partner]:
                    if reporter in eu_dic_II[year] and partner in eu_dic_II[year]:
                        eu_mat[reporter][partner][year] = 1
                    else:
                        eu_mat[reporter][partner][year] = 0

with open(path_raw_data+'/constructed_data/EU_bilat.csv', "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(('Reporter','Partner','year','EU'))

    for reporter in eu_mat:
        for partner in eu_mat[reporter]:
            for year in eu_mat[reporter][partner]:
                writer.writerow((reporter,partner,year,eu_mat[reporter][partner][year]))

In [15]:
eu_matrix = pd.read_csv(path_raw_data+'/constructed_data/EU_bilat.csv')

### I.2.2. CEPII
The CEPII data is already in the form of a dyadic matrix; we only need to decide which variables we want too keep

In [16]:
cepii_wb_map = {'Brunei': 'Brunei Darussalam', 'Cape Verde': 'Cabo Verde', "China, People's Republic of": 'China',
                'Egypt': 'Egypt, Arab Rep.', 'Hong Kong': 'Hong Kong SAR, China', 'Iran': 'Iran, Islamic Rep.', 'Kyrgyzstan': 'Kyrgyz Republic',
                'Laos': 'Lao PDR', 'Macau': 'Macao SAR, China', 'Russia': 'Russian Federation', 'Saint Kitts and Nevis': 'St. Kitts and Nevis',
                'Saint Lucia': 'St. Lucia', 'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines', 'Slovakia': 'Slovak Republic',
                'Syria': 'Syrian Arab Republic', 'Venezuela': 'Venezuela, RB', 'Yemen': 'Yemen, Rep.'}

In [17]:
#dtype option: we need to specify some (data) types of the columns because some of them contain multiple types (i.e. string and NaN)
#not specifying the data type will prompt Python to issue a warning but should not stop the code from running
cepii_matrix = pd.read_csv(path_raw_data+'/Gravity models/CEPII-Excel files/gravdata_cepii.csv', dtype={"iso2_o":str,"empire":str,"legold_o":str,"legnew_o":str})

#dropping year before 1990
cepii_matrix = cepii_matrix.loc[cepii_matrix['year']>=1990]
#keep only selected columns
cepii_matrix = cepii_matrix[["iso3_o", "iso3_d", "year", "contig", "comlang_off", "comrelig", "comcol", "col45", 
                             "fta_bb", "fta_hmr", "fta_wto"]]

#replace country codes in CEPII dataframe to reflect latest country code
cepii_matrix.replace(to_replace=["ROM", "ZAR", "TMP"], value=["ROU", "COD", "TLS"], inplace=True)

ctry_codes = pd.read_csv(path_raw_data+'/Gravity models/ISO-country_codes_II.csv')

#selection of the two columns in csv_data_II
ctry_codes = ctry_codes[['Common Name','ISO 3166-1 3 Letter Code']]

#by specifying left I am using keys of left data frame only
cepii_matrix = pd.merge(cepii_matrix,ctry_codes, how='left', left_on=['iso3_o'], right_on=['ISO 3166-1 3 Letter Code'])
cepii_matrix = pd.merge(cepii_matrix,ctry_codes, how='left', left_on=['iso3_d'], right_on=['ISO 3166-1 3 Letter Code'])

cepii_matrix.drop(["ISO 3166-1 3 Letter Code_x", "ISO 3166-1 3 Letter Code_y"], axis=1, inplace=True)
cepii_matrix.rename(columns={"Common Name_x":"Common_name_o", "Common Name_y":"Common_name_d"}, inplace=True)
cepii_matrix = cepii_matrix[["iso3_o", "Common_name_o", "iso3_d", "Common_name_d", "year", "contig", 'comlang_off', 'comrelig', 'comcol',
                             "col45", 'fta_hmr', 'fta_wto']] #'fta_bb', 

In [18]:
#drop iso3_o x iso3_d entries
cepii_matrix = cepii_matrix.loc[(cepii_matrix.iso3_o != cepii_matrix.iso3_d), :]
cepii_matrix = cepii_matrix.drop(["iso3_o", "iso3_d"], axis=1)
cepii_matrix.columns = ["Reporter", "Partner", "year", "contig", "comlang_off", "comrelig", "comcol", "col45", "fta_hmr", "fta_wto"]

cepii_matrix["Reporter"] = cepii_matrix["Reporter"].replace(to_replace=cepii_wb_map)
cepii_matrix["Partner"] = cepii_matrix["Partner"].replace(to_replace=cepii_wb_map)

# Extension of cepii matrix through 2018
cepii_matrix_2015 = cepii_matrix.loc[(cepii_matrix.year == 2015), :]

for y in [2016, 2017, 2018]:
    temp = cepii_matrix_2015
    temp.loc[:, "year"] = y
    
    cepii_matrix = pd.concat([cepii_matrix, temp])

/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/1457651485.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, "year"] = y
/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/1457651485.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, "year"] = y
/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/1457651485.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

### I.2.3. Bilateral assistance

In [19]:
odaI = pd.read_csv(path_raw_data+'/OECD/ODA/TABLE2A_05102020122645440.csv')
odaII = pd.read_csv(path_raw_data+'/OECD/ODA/TABLE2A_05102020122754846.csv')
oda = pd.concat([odaI,odaII])

oda = oda[oda['Aid type']=='Memo: ODA Total, Gross disbursements']
oda = oda[oda["Amount type"]=="Constant Prices"]

oda.drop_duplicates(inplace=True)

oda_tot = oda.groupby(by=["Recipient", "Year"]).sum()
oda_tot.drop(["DONOR", "RECIPIENT", "AIDTYPE", "PART", "TIME", "Flag Codes", "Flags", 'Reference Period Code', 'PowerCode Code', 'Reference Period'], 
             axis=1, inplace=True)
oda_tot.reset_index(inplace=True)
oda_tot.rename(columns={"Value":"ODA_tot"}, inplace=True)

oda = oda.merge(oda_tot, on=["Recipient", "Year"])

oda["ODA_share"] = oda.Value/oda.ODA_tot

oda_matrix = oda[["Recipient", "Donor", "Year", "ODA_share"]]
oda_matrix.columns = ["Reporter", "Partner", "year", "ODA_share"]

## I.3 Removal of 'self-distances' (and matrix normalization)

In [20]:
# Dictionary of all dyadic matrices
dyad_mat = [tradeflows_share["IMP"], tradeflows_share["EXP"], tradeflows_share["IMPEXP"], eu_matrix, cepii_matrix, oda_matrix]

for mat in dyad_mat:
    mat = mat.loc[mat.Reporter!=mat.Partner, :]
    
#Normalization - year by year! (see Lee & Yu, 2012)
#i = 0
#for mat in [eu_matrix]:#dyad_mat:
#    for yr in mat.Year.unique():
#        temp = mat.loc[mat.Year==yr, :]
#        temp.drop("Year", axis=1, inplace=True)
        
#        for col in temp.columns[2:]:
            #transform into matrix format
#            tempII = temp[["Reporter", "Partner", col]]
#            tempII = tempII.pivot(index='Reporter', columns='Partner')[col]
#            tempII = tempII.fillna(0)
#            tempII = tempII.to_numpy()

            #calculate eigenvalues; extract largest
#            w, v = LA.eig(tempII)
#            max_w = max(w)
            
            #divide values in orginal dataframe by that largest eigenvalue
#            mat.loc[mat.Year==yr, col] = mat.loc[mat.Year==yr, col]/max_w
#    i+=1

# III. Covariates

## III.1 Policy information/experience diffusion

Diffusion channels: EU, trade, cultural similarities, official development aid

In [21]:
policy_indicators = ["tax", "ets", "fit", "rps", "pricing", "techpol"]

In [22]:
df_policy = policies[["Country", "year"]]
df_policy.columns = ["Reporter", "year"]

for mat in dyad_mat:

    dyad_mat_pol = mat.merge(policies, left_on=["Partner", "year"], right_on=["Country", "year"], how='left')
    dyad_mat_pol.drop("Country", axis=1, inplace=True)
    
    for var in policy_indicators:
        weights = [x for x in list(mat.columns) if x not in ["Reporter", "Partner", "year", "FLOW"]]
        for col in weights:
            dyad_mat_pol[var+"_"+col] = dyad_mat_pol[var]*dyad_mat_pol[col]
            dyad_mat_pol[var+"_"+col] = dyad_mat_pol[var+"_"+col].astype(float)
    
    dyad_mat_pol.drop(policy_indicators, axis=1, inplace=True)
    dyad_mat_pol.drop(weights, axis=1, inplace=True)
    dyad_mat_pol = dyad_mat_pol.groupby(by=["Reporter", "year"]).sum()
    dyad_mat_pol.reset_index(inplace=True)
    
    df_policy = df_policy.merge(dyad_mat_pol, on=["Reporter", "year"], how="left")

In [23]:
df_policy.columns = ['Reporter', 'year', 
                     'tax_imp', 'ets_imp', 'fit_imp', 'rps_imp', 'pricing_imp', 'techpol_imp',
                     'tax_exp', 'ets_exp', 'fit_exp', 'rps_exp', 'pricing_exp', 'techpol_exp',
                     'tax_impexp', 'ets_impexp', 'fit_impexp', 'rps_impexp', 'pricing_impexp', 'techpol_impexp',
                     'tax_eu', 'ets_eu', 'fit_eu', 'rps_eu', 'pricing_eu', 'techpol_eu',
                     'tax_contig', 'tax_comlang_off', 'tax_comrelig', 'tax_comcol', 'tax_col45', 'tax_fta_hmr', 'tax_fta_wto',
                     "ets_contig", 'ets_comlang_off', 'ets_comrelig', 'ets_comcol', 'ets_col45', 'ets_fta_hmr', 'ets_fta_wto',
                     "fit_contig", 'fit_comlang_off', 'fit_comrelig', 'fit_comcol', 'fit_col45', 'fit_fta_hmr', 'fit_fta_wto',
                     "rps_contig", 'rps_comlang_off', 'rps_comrelig', 'rps_comcol', 'rps_col45', 'rps_fta_hmr', 'rps_fta_wto',
                     "pricing_contig", 'pricing_comlang_off', 'pricing_comrelig', 'pricing_comcol', 'pricing_col45', 'pricing_fta_hmr', 'pricing_fta_wto',
                     "techpol_contig", 'techpol_comlang_off', 'techpol_comrelig', 'techpol_comcol', 'techpol_col45', 'techpol_fta_hmr', 'techpol_fta_wto',
                     'tax_ODA', 'ets_ODA', 'fit_ODA', 'rps_ODA', 'pricing_ODA', 'techpol_ODA']

## III.2 Learning from success
Difference between average power sector CO2 emissions (per capita) of countries with and without policy

In [36]:
co2_power = co2_em_elec
population = wb_form[["Country", "year", "pop_tot"]]

co2_power_pc = co2_power.merge(population, on=["Country", "year"])
co2_power_pc["co2_power_pc"] = co2_power_pc.co2em*1000/co2_power_pc.pop_tot

success = co2_power_pc.merge(policies, on=["Country", "year"])

for policy in ["fit"]:
    temp = success.groupby(by=[policy, "year"]).mean()
    temp.reset_index(inplace=True)
    temp = temp[["fit", "year", "co2_power_pc"]]

    temp_0 = temp.loc[temp.fit==0]
    temp_1 = temp.loc[temp.fit==1]
    
    temp = temp_0.merge(temp_1, on=["year"])
    temp["co2_power_pc_diff"] = temp.co2_power_pc_x - temp.co2_power_pc_y

## III.3 Technology

1. Diffusion channels: EU, trade

### III.3.1 Dyadic weighted knowledge and technology stock

In [25]:
dyad_tech = [tradeflows_share["IMP"], tradeflows_share["EXP"], eu_matrix]

df_tech = policies[["Country", "year"]]
df_tech.columns = ["Reporter", "year"]

for mat in dyad_tech:
    for df in [patents_cum, recap_cum]:
        dyad_mat_tech = mat.merge(df, left_on=["Partner", "year"], right_on=["Country", "year"])
        dyad_mat_tech.drop("Country", axis=1, inplace=True)
        
        for var in list(df.columns)[2:]:
            weights = [x for x in list(mat.columns) if x not in ["Reporter", "Partner", "year", "FLOW"]]
            for col in weights:
                dyad_mat_tech[var+"_"+col] = dyad_mat_tech[var]*dyad_mat_tech[col]

        dyad_mat_tech.drop(list(df.columns)[2:], axis=1, inplace=True)
        dyad_mat_tech.drop(weights, axis=1, inplace=True)
        dyad_mat_tech = dyad_mat_tech.groupby(by=["Reporter", "year"]).sum()
        dyad_mat_tech.reset_index(inplace=True)

        df_tech = df_tech.merge(dyad_mat_tech, on=["Reporter", "year"], how="left")

#rename columns
df_tech.rename(columns=dict(zip(['patents_ds_trade_flow_share_x', 'capacity_ws_ds_trade_flow_share_x', 'patents_ds_trade_flow_share_y',
                                  'capacity_ws_ds_trade_flow_share_y', 'patents_ds_EU','capacity_ws_ds_EU'],
                                 ['patents_ds_imp', 'capacity_ws_ds_imp', 'patents_ds_exp', 'capacity_ws_ds_exp', 'patents_ds_eu', 
                                  'capacity_ws_ds_eu'])), inplace=True)


### III.3.2 Country-specific global knowledge and technology stocks

In [26]:
stock_re = recap_cum[["Country", "year"]]
stock_pat = patents_cum[["Country", "year"]]

for ctry in recap_cum.Country.unique():
    temp = recap_cum.loc[recap_cum.Country!=ctry, :].groupby("year").sum()
    
    stock_re.loc[stock_re.Country==ctry, "capacity_ds_global"] = np.array(temp.iloc[:, 0])

for ctry in patents_cum.Country.unique():
    temp = patents_cum.loc[patents_cum.Country!=ctry, :].groupby("year").sum()

    stock_pat.loc[stock_pat.Country==ctry, "patents_ds_global"] = np.array(temp.iloc[:, 0])

global_stocks = stock_re.merge(stock_pat, on=["Country", "year"], how="outer") #merge on outer otherwise it will keep only common keys

### III.3.3 Stocks x trade level (% GDP)

In [37]:
imports = wb_wdi.loc[wb_wdi["Series Name"]=="Imports of goods and services (% of GDP)", :].copy()
imports.drop("Series Name", axis=1, inplace=True)
imports.rename(columns={"value":"imports_gdp", "Country":"Reporter"}, inplace=True)

df_tech = df_tech.merge(imports, on=["Reporter", "year"], how="left")
df_tech["patents_ds_imp_imp"] = df_tech.patents_ds_imp*df_tech.imports_gdp
df_tech["capacity_ws_ds_imp_imp"] = df_tech.capacity_ws_ds_imp*df_tech.imports_gdp

df_tech.drop("imports_gdp", axis=1, inplace=True)

## III.4 Leakage risk index (foreign policy stringency)
- Arguably, can be measured with respect to imports and exports. For now, focus on imports. But one could imagine building a variable that captures the policy stringency of trade competitors, as in Simmons and Elkins.
- Question: the leakage risk should be measured with regard to the entire set of policies targeting the power sector, not policy by policy...?! ==> construct a composite index similar to Botta & Kozluk?

In [28]:
# Binary measures (all policies)
# (capturing only legally binding FiTs/targets): either policy is in place or not

co2_int_power = pd.read_csv(path_raw_data+"/IEA/IEA_GHG_int_elec.csv")
co2_int_power = co2_int_power.loc[(co2_int_power.PRODUCT=="ADPRODUCT001") & (co2_int_power.FLOW=="ADFLOW001"), ["Country", "Time", "Value"]]
co2_int_power = co2_int_power.loc[~(co2_int_power.Country.str.match("Memo|OECD|Non|World|Other")), :]
co2_int_power = co2_int_power.replace(to_replace=map_iea_wb)

df_leak = policies[["Country", "year"]]

for mat in [tradeflows_share["IMP"]]:
    dyad_mat_leak = mat.merge(co2_int_power, left_on=["Partner", "year"], right_on=["Country", "Time"], how='left')
    dyad_mat_leak = dyad_mat_leak.merge(policies, left_on=["Partner", "year"], right_on=["Country", "year"], how='left')
    dyad_mat_leak = dyad_mat_leak.drop(["Country_x", "Country_y"], axis=1)
    dyad_mat_leak.rename(columns={"Reporter":"Country"}, inplace=True)
    
    for var in policy_indicators:
        weights = list(mat.columns)[4:]
    
        for col in weights:
            dyad_mat_leak[var+"_"+"leakage_index"] = dyad_mat_leak[col]*dyad_mat_leak.Value*dyad_mat_leak[var]

    dyad_mat_leak.drop(policy_indicators+weights+["Time", "Value"], axis=1, inplace=True)
    dyad_mat_leak = dyad_mat_leak.groupby(["Country", "year"]).sum()
    dyad_mat_leak.reset_index(inplace=True)
    
    df_leak = df_leak.merge(dyad_mat_leak, on=["Country", "year"], how="left")
    
# Continuous measures
# Carbon pricing mechanisms: carbon price
# RPS - % of RE to be procured (OECD, environmental policy stringency index)


## III.4. Mean global policy

In [29]:
#Continuous measures
#Share of country's power sector CO2 emissions in World total power sector emissions
em_power = pd.read_csv("/Users/gd/OneDrive - rff/Documents/Research/projects/ecp/ecp_dataset/source_data/ghg_inventory/raw/national/IEA/iea_energy_co2_emissions/detailed_figures/agg_product/iea_aggprod_WBnames.csv")
em_power = em_power.loc[(em_power.Flow=="ABFLOW003") & (em_power.Product=="Total"), ['Country', 'year', 'Flow', 'Product', 'CO2_emissions']]
em_power = em_power.drop(["Product"], axis=1)
               
em_power = em_power.replace(to_replace=map_iea_wb)

em_power_wld = em_power.loc[em_power.Country=="World", :]
em_power_wld = em_power_wld.rename(columns={"CO2_emissions":"CO2_emissions_wld"})
em_power_wld = em_power_wld.drop(["Country"], axis=1)
em_power = em_power.merge(em_power_wld, on=["year", "Flow"], how='left')
em_power = em_power.loc[em_power.Country!="World", :]

em_power.loc[:, "CO2_share"] = em_power.loc[:, "CO2_emissions"]/em_power.loc[:, "CO2_emissions_wld"]

em_power = em_power.drop(["Flow", "CO2_emissions", "CO2_emissions_wld"], axis=1)

free_riding = em_power.merge(policies, on=["Country", "year"], how='right')

for col in policy_indicators:
    free_riding.loc[:, col+"_share"] = free_riding.loc[:, col]*free_riding.loc[:, "CO2_share"]

free_riding = free_riding.drop(policy_indicators+["CO2_share"], axis=1)
free_riding = free_riding.groupby(["year"]).sum()
free_riding = free_riding.reset_index()                

temp = policies[["Country", "year"]].merge(free_riding, on=["year"], how='left')
free_riding = temp

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [30]:


#Binary measures (all policies)
# Total number of active schemes / total number of countries (Simmons & Elkins) - excluding own policy (different from Simons & Elkins)

mean_policy_power = policies[["Country", "year"]]
col_names = ["tax_avg", "ets_avg", "fit_avg", "rps_avg", "pricing_avg", "techpol_avg"]

for col in col_names:
    mean_policy_power[col] = np.nan

for ctry in policies.Country.unique():
    temp = policies.loc[policies.Country!=ctry, :].copy()
    temp = temp.groupby("year").mean()
    
    i = 0
    for col in col_names:
        mean_policy_power.loc[mean_policy_power.Country==ctry, col] = np.array(temp.iloc[:, i])
        i += 1

mean_policy_power["clim_pol_avg"] = (mean_policy_power.tax_avg+mean_policy_power.ets_avg+mean_policy_power.fit_avg+mean_policy_power.rps_avg)/4


/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/634434391.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_policy_power[col] = np.nan
/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/634434391.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_policy_power[col] = np.nan
/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_46984/634434391.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

# IV. Control mechanisms
The ECP jurisdiction-year pairs should serve as reference for the panel

### Domestic knowledge stock

In [31]:
#patents_cum - calculated above

### Largest innovators identifier

In [32]:
inno_2016 = patents_cum.loc[(patents_cum.year==2016)]

#number of countries in sample
#len(inno_2016)

#inno_2016[pd.qcut(inno_2016['patents_ds'], 10, labels=range(10)).eq(9)]
inno_2016_quant = inno_2016.loc[inno_2016['patents_ds'] > inno_2016['patents_ds'].quantile(0.9), :]

inno_ctries = inno_2016_quant.Country.unique()

inno_bin = policies[["Country", "year"]].copy()
inno_bin["innovator"] = 0
inno_bin.loc[inno_bin.Country.isin(inno_ctries), "innovator"] = 1

#inno_2016_quant.sort_values(by="patents_ds")

### Economic conditions (World Bank)

In [81]:
#WB WDI

wb_wdi = pd.read_csv(path_raw_data+"/WB_WDI/WB_data.csv")
wb_wdi.drop(["Country Code", "Series Code"], axis=1, inplace=True)

wb_wdi = wb_wdi.melt(id_vars=["Country Name", "Series Name"])

wb_wdi.rename(columns={"Country Name":"Country", "variable":"year"}, inplace=True)

wb_wdi["year"] = wb_wdi["year"].apply(lambda x: x[:4])
wb_wdi["year"] = wb_wdi["year"].astype(int)

series_form = {"merch_exp":'Merchandise exports (current US$)',
               "gdp_pc_ppp":"GDP, PPP (constant 2017 international $)",
               "exports":"Exports of goods and services (% of GDP)",
               "imports":"Imports of goods and services (% of GDP)",
               "pop_tot":"Population, total",
               "elec_oil":"Electricity production from oil sources (% of total)",
               "elec_gas":"Electricity production from natural gas sources (% of total)",
               "elec_coal":"Electricity production from coal sources (% of total)",
               "co2_tot":"CO2 emissions (kt)",
               "co2_pc":"CO2 emissions (metric tons per capita)",
               "co2_int_gdp":"CO2 emissions (kg per 2017 PPP $ of GDP)"}

wb_form = pd.DataFrame()

for series in series_form.keys():
    temp = wb_wdi[wb_wdi["Series Name"]==series_form[series]].copy()
    temp.drop("Series Name", axis=1, inplace=True)
    temp.rename(columns={"value":series},
                inplace=True)

    if wb_form.empty:
        wb_form = temp
    else:
        wb_form = pd.merge(wb_form, temp, on=["Country", "year"], how='left')


In [88]:
merchExp = wb_form[["Country", "year", "merch_exp"]]

merchExpWld = wb_form[["Country", "year", "merch_exp"]].groupby(["year"]).sum()
merchExpWld.reset_index(inplace=True)

merchExp = pd.merge(merchExp, merchExpWld, on=["year"], how='left')
merchExp["merch_exp"] = merchExp.merch_exp_x/merchExp.merch_exp_y
merchExp.drop(["merch_exp_x", "merch_exp_y"], axis=1, inplace=True)

wb_form.drop(["merch_exp"], axis=1, inplace=True)

wb_form = wb_form.merge(merchExp, on=["Country", "year"], how='left')


In [41]:
# export as share of world export

exportsCO2int = concatenate("/Users/gd/OneDrive - rff/Documents/Research/projects/climate_policy_adoption/cp_diffusion/data/raw_data/bilateral_trade/WB_(UNCOMTRADE)/raw/co2Intexports")

# aggregate at reporter-partner level (i.e., removing product level)
exportsCO2int = exportsCO2int.groupby(['Nomenclature', 'ReporterISO3', 'ReporterName',
                                        'PartnerISO3', 'PartnerName', 'Year', 'TradeFlowName', 'TradeFlowCode']).sum()

exportsCO2int.reset_index(inplace=True)

# remove 'world' rows from `exportsCO2int` dataframe
exportsCO2int = exportsCO2int.loc[exportsCO2int.PartnerISO3!="WLD"]

# create dataframe with total world exports
exportsCO2intWLD = exportsCO2int.groupby(['Nomenclature', 'Year', 
                                             'TradeFlowName', 'TradeFlowCode']).sum()
exportsCO2intWLD.reset_index(inplace=True)
exportsCO2intWLD.rename(columns={"TradeValue in 1000 USD":"TradeValue in 1000 USD_WLD"},
                        inplace=True)

# aggregate at reporter level
exportsCO2int = exportsCO2int.groupby(['Nomenclature', 'ReporterISO3', 'ReporterName',
                                       'Year', 'TradeFlowName', 'TradeFlowCode']).sum()
exportsCO2int.reset_index(inplace=True)

exportsCO2int = exportsCO2int.merge(exportsCO2intWLD, on=['Nomenclature', 'Year', 
                                                          'TradeFlowName', 'TradeFlowCode'])
exportsCO2int.drop(["ProductCode_x", "ProductCode_y"], axis=1,
                   inplace=True)

exportsCO2int["co2intExpShare"] = exportsCO2int["TradeValue in 1000 USD"]/exportsCO2int["TradeValue in 1000 USD_WLD"]

exportsCO2int.drop(['Nomenclature', 'ReporterISO3', 'TradeFlowName',
                    'TradeFlowCode', 'TradeValue in 1000 USD', 'TradeValue in 1000 USD_WLD'], 
                   axis=1, inplace=True)
exportsCO2int.rename(columns={"ReporterName":"Country", "Year":"year"},
                     inplace=True)

exportsCO2int = exportsCO2int.sort_values(by=["Country", "year"])

### Power sector characteristics

In [34]:
eia = pd.read_csv(path_raw_data+"/EIA/EIA_FF_gen.csv", skiprows=[0], encoding='latin-1')
co2_em_elec = pd.read_csv(path_raw_data+"/IEA/IEA_CO2_elec.csv", encoding="latin-1")

co2_em_elec = co2_em_elec[["Country", "TIME", "Value"]]
co2_em_elec.rename(columns={"TIME":"year", "Value":"co2em"}, inplace=True)

#EIA
eia_ctry = np.array(eia.loc[eia.API.isna(), "Unnamed: 1"])

eia_tot = pd.DataFrame(eia.loc[eia["Unnamed: 1"]=="    Generation (billion kWh)", :])
eia_tot["Country"] = eia_ctry
eia_tot.drop(["API", "Unnamed: 1"], axis=1, inplace=True)
eia_tot = eia_tot.melt(id_vars="Country")
eia_tot.rename(columns={"variable":"Year", "value":"Tot_gen_bkWh"}, inplace=True)

eia_ff = pd.DataFrame(eia.loc[eia["Unnamed: 1"]=="        Fossil fuels (billion kWh)", :])
eia_ff["Country"] = eia_ctry
eia_ff.drop(["API", "Unnamed: 1"], axis=1, inplace=True)
eia_ff = eia_ff.melt(id_vars="Country")
eia_ff.rename(columns={"variable":"Year", "value":"FF_gen_bkWh"}, inplace=True)

eia_form = eia_ff.merge(eia_tot, on=["Country", "Year"])

eia_form.rename(columns={"Year":"year"}, inplace=True)

eia_wb_map = {'The Bahamas': 'Bahamas, The', 'Brunei': 'Brunei Darussalam', 'Burma': 'Myanmar', 'Congo-Brazzaville': 'Congo, Rep.',
              'Congo-Kinshasa': 'Congo, Dem. Rep.', 'Côte d\x92Ivoire': "Cote d'Ivoire", 'Egypt': 'Egypt, Arab Rep.',
              'Hong Kong': 'Hong Kong SAR, China', 'Iran': 'Iran, Islamic Rep.', 'Kyrgyzstan': 'Kyrgyz Republic', 'Laos': 'Lao PDR',
              'Macau': 'Macao SAR, China', 'North Korea': 'Korea, Dem. Rep.', 'Russia': 'Russian Federation',
              'Saint Kitts and Nevis': 'St. Kitts and Nevis', 'Saint Lucia': 'St. Lucia',
              'Saint Vincent/Grenadines': 'St. Vincent and the Grenadines', 'Slovakia': 'Slovak Republic', 'South Korea': 'Korea, Rep.',
              'Syria': 'Syrian Arab Republic', 'Venezuela': 'Venezuela, RB', 'Yemen': 'Yemen, Rep.'}

iea_wb_map = {'Bolivarian Republic of Venezuela': 'Venezuela, RB', "Côte d'Ivoire": "Cote d'Ivoire", 'Egypt': 'Egypt, Arab Rep.',
              "Democratic People's Republic of Korea": 'Korea, Dem. Rep.', 'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
              'Hong Kong (China)': 'Hong Kong SAR, China', 'Islamic Republic of Iran': 'Iran, Islamic Rep.', 'Korea': 'Korea, Rep.',
              'Kyrgyzstan': 'Kyrgyz Republic', "Lao People's Democratic Republic": 'Lao PDR', 'Myanmar': 'Myanmar',
              "People's Republic of China": 'China', 'Plurinational State of Bolivia': 'Bolivia', 'Republic of Moldova': 'Moldova',
              'Republic of North Macedonia': 'North Macedonia', 'Republic of the Congo': 'Congo, Rep.',
              'United Republic of Tanzania': 'Tanzania', 'Viet Nam': 'Vietnam', 'Yemen': 'Yemen, Rep.'}

eia_form.replace(to_replace=eia_wb_map, inplace=True)
co2_em_elec.replace(to_replace=iea_wb_map, inplace=True)

eia_form["year"] = eia_form["year"].astype(int)

In [35]:
power = eia_form.merge(co2_em_elec, on=["Country", "year"], how="left")

power.loc[power.FF_gen_bkWh=="--", "FF_gen_bkWh"] = np.nan
power.loc[power.Tot_gen_bkWh=="--", "Tot_gen_bkWh"] = np.nan

power["FF_gen_bkWh"] = power["FF_gen_bkWh"].astype(float)
power["Tot_gen_bkWh"] = power["Tot_gen_bkWh"].astype(float)
power["ff_perc_tot"] = power.FF_gen_bkWh/power.Tot_gen_bkWh
power["co2_int"] = power.co2em/power.Tot_gen_bkWh

power["ffperctot_co2int"] = power.ff_perc_tot*power.co2_int

power.drop(["FF_gen_bkWh", "Tot_gen_bkWh", "co2em"], axis=1, inplace=True)

power = power.loc[(power.year>=1990) & (power.year<=2018), :]

### Institutions (Varieties of Democracy)

In [42]:
#VDEM
vdem = pd.read_csv('/Users/gd/OneDrive - rff/Documents/Research/resources/data/political_institutions/VDem/Country_Year_V-Dem_Core_CSV_v10/V-Dem-CY-Core-v10.csv')
vdem = vdem[["country_name", "year", 'v2x_polyarchy', 'v2x_libdem']]

vdem.rename(columns={"country_name":"Country"}, inplace=True)

In [43]:
vdem_wb_map = {'Burma/Myanmar': 'Myanmar', 'Cape Verde': 'Cabo Verde', 'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
               'Egypt': 'Egypt, Arab Rep.', 'Hong Kong': 'Hong Kong SAR, China', 'Iran': 'Iran, Islamic Rep.', 'Ivory Coast': "Cote d'Ivoire",
               'Kyrgyzstan': 'Kyrgyz Republic', 'Laos': 'Lao PDR', 'North Korea': 'Korea, Dem. Rep.', 'Republic of the Congo': 'Congo, Rep.',
               'Russia': 'Russian Federation', 'Slovakia': 'Slovak Republic', 'South Korea': 'Korea, Rep.', 'Syria': 'Syrian Arab Republic',
               'The Gambia': 'Gambia, The', 'United States of America': 'United States', 'Venezuela': 'Venezuela, RB', 'Yemen': 'Yemen, Rep.'}

vdem.replace(to_replace=vdem_wb_map, inplace=True)

### Duration

In [44]:
duration = policies.copy()
duration = duration[["Country", "year"]]

for ctry in duration.Country.unique():
    i = 1 #start at one because some hazard models in the lifelines library can't deal with non-positive duration values
    for yr in duration.year.unique():
        duration.loc[(duration.Country==ctry) & (duration.year==yr), "Duration"] = i
        i += 1

# VI. Dataset

In [93]:
db = policies.copy()

i = 0

for data in [df_policy, df_tech, patents_cum, df_leak, free_riding, global_stocks, power, inno_bin, duration, wb_form, exportsCO2int, vdem]:
    data.rename(columns={"Reporter":"Country"}, inplace=True)
    db = db.merge(data, on=["Country", "year"], how='left')
    i+=1
    
#identifier column
db['UniqueID'] = db['Country'].astype(str)+'_'+db['year'].astype(str)

#keeping only observations between 1990 and 2016 (last year for which patent data is available)
db = db.loc[(db['year'] >= 1990) & (db['year'] <= 2018), :]

### Normalization of continuous, non-interacted, variables

In [94]:
db_main = db.copy()

norm_var = ['gdp_pc_ppp', 'pop_tot', 'co2_int', 'co2_int_gdp', 'ff_perc_tot', 'v2x_polyarchy']

for col in norm_var:
    db_main[col+"_sc"] = db_main[col]/db_main[col].mean()*10-10

### Scaling of patents variables and renewable capacity

In [95]:
scale_col = ['patents_ds_imp', 'patents_ds_imp_imp', 'patents_ds_exp', 'patents_ds_global', 'patents_ds_eu', 'patents_ds']

for col in scale_col:
    db_main[col] = db_main[col]/100

for col in ["capacity_ds_global"]:
    db_main[col] = db_main[col]/1000

### Full dataset (for probit)

In [96]:
ctry_drop = ["Andorra", "Djibouti", "Ethiopia", "Fiji", "Korea, Dem. Rep.", "Lesotho", "Liechtenstein", "Monaco", "Puerto Rico", 
             "Papua New Guinea", "San Marino", "Serbia", "Montenegro", "Somalia", "South Sudan", "Suriname", "West Bank and Gaza", "Sao Tome and Principe",
             "St. Kitts and Nevis", "St. Lucia", "St. Vincent and the Grenadines", "Timor-Leste", "Trinidad and Tobago"]

db_main = db_main.loc[~db_main.Country.isin(ctry_drop), :]

#drop NA values
db_main.dropna(subset=['tax', 'ets', 'fit', 'rps', 'pricing', 'techpol','tax_imp', 'ets_imp', 'fit_imp', 'rps_imp', 'pricing_imp', 'techpol_imp',
                       'tax_impexp', 'ets_impexp', 'fit_impexp', 'rps_impexp', 'pricing_impexp', 'techpol_impexp',
                       'tax_comlang_off', 'tax_comrelig', 'tax_comcol', 'tax_col45', 'tax_fta_hmr',
                       'ets_comlang_off', 'ets_comrelig', 'ets_comcol', 'ets_col45', 'ets_fta_hmr', 
                       'fit_comlang_off', 'fit_comrelig', 'fit_comcol', 'fit_col45', 'fit_fta_hmr',
                       'rps_comlang_off', 'rps_comrelig', 'rps_comcol', 'rps_col45', 'rps_fta_hmr',
                       'pricing_comlang_off', 'pricing_comrelig', 'pricing_comcol', 'pricing_col45', 'pricing_fta_hmr',
                       'techpol_comlang_off', 'techpol_comrelig', 'techpol_comcol', 'techpol_col45', 'techpol_fta_hmr',
                       'patents_ds_imp', 'capacity_ws_ds_imp', 'patents_ds_imp_imp', 'capacity_ws_ds_imp_imp',
                       #'tax_avg', 'ets_avg', 'fit_avg', 'rps_avg',
                       'imports', 'gdp_pc_ppp', 'ff_perc_tot', 'ffperctot_co2int', 'v2x_polyarchy'], inplace=True) #, 

#restrict to 101 countries for which we have patent info
#pat_ctry = ks_pat.Country.unique()
#db_main = db_main.loc[db_main.Country.isin(pat_ctry), :]

db_main.to_csv(path_cp_diff+'/CPdiff_dataset_v4.csv',index=None)

### Datasets for hazard regressions
#### Tax, ETS, FiT, RPS

In [97]:
#Dataset where only the first observation post implementation is kept

dummies = ['tax', 'ets', 'fit', 'rps', 'pricing', 'techpol']

for policy in dummies:
    db_hazard_0 = db_main.loc[db[policy]==0, :].copy()
    db_hazard_1 = db_main.loc[db[policy]==1, :].copy()

    #Sorting dataframe - VERY important that the dataframe be sorted by:'jurisdiction' then 'year' otherwise the next
    #command will simply keep the first entry which might well be a different year than the first in which a scheme was
    #introduced
    db_hazard_1.sort_values(by=['Country','year'],inplace=True)
    db_hazard_1.drop_duplicates(subset=['Country', policy],keep='first',inplace=True)

    db_hazard = pd.concat([db_hazard_0, db_hazard_1])

    #drop dummy of other policies
    drop_list = [x for x in dummies if x != policy]
    db_hazard.drop(drop_list, axis=1, inplace=True)
    
    #also drop the covariates that pertain to other policies
    discard = []
    
    drop_col = dummies.copy()
    drop_col.remove(policy)
    
    for col in drop_col:
        temp_list = [x for x in db_hazard.columns if x.startswith(col) == True]
        discard.extend(temp_list)
    
    covariates = [x for x in db_hazard.columns if x not in discard]

    db_hazard = db_hazard[covariates]
    
    #rename columns to remove reference to policy
    colNames = [x for x in db_hazard.columns if x.startswith(policy) == True]

    for col in colNames:
        db_hazard.rename(columns={col:"policy"+col[len(policy):]}, inplace=True)
        
    db_hazard.to_csv(path_cp_diff+'/CPdiff_hz_'+policy+'_v4.csv',index=None)
